In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import sklearn
from sklearn.model_selection import train_test_split
from random import sample
import os,gc,re
import warnings
import sys
if not sys.warnoptions:
    warnings.simplefilter("ignore")
import os
# os.environ["TOKENIZERS_PARALLELISM"] = "false"
%env WANDB_DISABLED=True 

In [ ]:
import uuid

In [ ]:
import transformers

In [ ]:
listid = []
for i in range(9990):
    listid.append(str(uuid.uuid4()))

In [ ]:
import sentencepiece

In [ ]:
train   = pd.read_csv('../input/chaii-hindi-and-tamil-question-answering/train.csv')
trainhi = pd.read_csv('../input/en-to-ta-qa-dataset-2-14-rows/en_to_ta_2**14_rows/mlqa_hindi.csv')
trainta = pd.read_csv('../input/squad-translated-to-tamil-for-chaii/squad_translated_tamil.csv')
trainta2 =pd.read_csv('../input/en-to-ta-qa-dataset-2-14-rows/en_to_ta_2**14_rows/tamil_extra_translated_data.csv')
test   = pd.read_csv('../input/chaii-hindi-and-tamil-question-answering/test.csv')
#submission= pd.read_csv('./sample_submission.csv')
#    return train,test,submission
trainta['language']='tamil'
tam_2 = trainta2.drop(['en_context','en_answer_text','en_question'],1)
tam_2['language']='tamil'
tam_2.columns=['context','question','answer_text','answer_start','language']

In [ ]:
import random
import torch
manualSeed = 42
np.random.seed(manualSeed)
random.seed(manualSeed)
torch.manual_seed(manualSeed)
# if you are suing GPU
torch.cuda.manual_seed(manualSeed)
torch.cuda.manual_seed_all(manualSeed)
torch.backends.cudnn.enabled = False 
torch.backends.cudnn.benchmark = False
torch.backends.cudnn.deterministic = True

In [ ]:
def get_answers(x):
    start = x[0]
    text = x[1]
    return {
        'answer_start': [start],
        'text': [text]
    }

train['answers'] = train[['answer_start', 'answer_text']].apply(get_answers, axis=1)
trainhi['answers'] = trainhi[['answer_start', 'answer_text']].apply(get_answers, axis=1)
trainta['answers'] = trainta[['answer_start', 'answer_text']].apply(get_answers, axis=1)
tam_2['answers'] = tam_2[['answer_start', 'answer_text']].apply(get_answers, axis=1)
#pd.set_option('display.max_colwidth',None)

In [ ]:
additional_train = pd.concat([trainhi,trainta,tam_2])
additional_train.insert(loc=0,column='id',value =listid)
test.head(1)
# len(additional_train)

In [ ]:
train = pd.concat([train,additional_train])

In [ ]:
from transformers import AutoTokenizer, AutoModelForQuestionAnswering
model_name = '../input/chaiicustom-model/chaii2'
tokenizer = AutoTokenizer.from_pretrained(model_name)
#model = AutoModelForQuestionAnswering.from_pretrained("bert-large-uncased-whole-word-masking-finetuned-squad")

In [ ]:
import transformers
assert isinstance(tokenizer, transformers.PreTrainedTokenizerFast)
import datasets
from datasets import Dataset

In [ ]:
# cnt = 0;
# drop_d= []
# for i in range(9990):
#     if len(train['context'].iloc[i].split())>400:
#         cnt = cnt+1 
#         drop_d.append(i) 

In [ ]:
train.reset_index(inplace=True)
train.index

In [ ]:
ylang = train['language']
xtrain = train.drop(columns=['language'],axis=0)

In [ ]:
x_train, x_valid, y_train, y_valid = train_test_split(xtrain, ylang,
                                                    test_size=0.1,
                                                    random_state=0,
                                                    stratify=ylang)

In [ ]:
y1train = pd.DataFrame(y_train,columns=['language'])
y1valid = pd.DataFrame(y_valid,columns=['language'])

In [ ]:
df_train= pd.concat([x_train,y1train],axis=1)
df_valid= pd.concat([x_valid,y1valid],axis=1)

In [ ]:

cnt = 0;
drop_d= []
for i in range(9990):
    if len(df_train['context'].iloc[i].split())>500:
        cnt = cnt+1 
        drop_d.append(i) 
     
print(cnt)
print(len(drop_d))

In [ ]:
df_train.reset_index(inplace=True)
df_train.index

In [ ]:
df_train.drop(list(drop_d), inplace=True)
df_train.reset_index(drop = True,inplace=True)
df_train.index

In [ ]:
df_valid.reset_index(drop = True,inplace=True)
df_valid.index

In [ ]:
df_train.columns
df_valid.columns

In [ ]:
df_train=df_train.drop(['index'],axis=1)
df_valid=df_valid.drop(['index'],axis=1)

In [ ]:
df_train.columns
df_valid.columns

In [ ]:
col_list = list(df_train.columns)
col_list[5],col_list[6]= col_list[6],col_list[5]
df_train=df_train[col_list]
df_train['answers'].head()
df_train.columns

In [ ]:
col_list = list(df_valid.columns)
col_list[5],col_list[6]= col_list[6],col_list[5]
df_valid=df_valid[col_list]
df_valid.columns

In [ ]:
df_valid['answers'].head()

In [ ]:
pad_on_right = tokenizer.padding_side == "right"
df_train.columns

In [ ]:
max_length = 384
doc_stride=128
train_dataset = Dataset.from_pandas(df_train)
valid_dataset = Dataset.from_pandas(df_valid)

In [ ]:
def prepare_train_features(examples):
    tokenized_examples = tokenizer(
        examples["question" if pad_on_right else "context"],
        examples["context" if pad_on_right else "question"],
        truncation="only_second" if pad_on_right else "only_first",
        max_length=max_length,
        stride=doc_stride,
        return_overflowing_tokens=True,
        return_offsets_mapping=True,
        padding="max_length",
    )
    # Since one example might give us several features if it has a long context, we need a map from a feature to
    # its corresponding example. This key gives us just that.
    sample_mapping = tokenized_examples.pop("overflow_to_sample_mapping")
    # The offset mappings will give us a map from token to character position in the original context. This will
    # help us compute the start_positions and end_positions.
    offset_mapping = tokenized_examples.pop("offset_mapping")

    # Let's label those examples!
    tokenized_examples["start_positions"] = []
    tokenized_examples["end_positions"] = []

    for i, offsets in enumerate(offset_mapping):
        # We will label impossible answers with the index of the CLS token.
        input_ids = tokenized_examples["input_ids"][i]
        cls_index = input_ids.index(tokenizer.cls_token_id)

        # Grab the sequence corresponding to that example (to know what is the context and what is the question).
        sequence_ids = tokenized_examples.sequence_ids(i)

        # One example can give several spans, this is the index of the example containing this span of text.
        sample_index = sample_mapping[i]
        answers = examples["answers"][sample_index]
       # print(answers)
        # If no answers are given, set the cls_index as answer.
        if len(answers["answer_start"]) == 0:
            tokenized_examples["start_positions"].append(cls_index)
            tokenized_examples["end_positions"].append(cls_index)
        else:
            # Start/end character index of the answer in the text.
            start_char = answers["answer_start"][0]
            end_char = start_char + len(answers["text"][0])

            # Start token index of the current span in the text.
            token_start_index = 0
            while sequence_ids[token_start_index] != (1 if pad_on_right else 0):
                token_start_index += 1

            # End token index of the current span in the text.
            token_end_index = len(input_ids) - 1
            while sequence_ids[token_end_index] != (1 if pad_on_right else 0):
                token_end_index -= 1

            # Detect if the answer is out of the span (in which case this feature is labeled with the CLS index).
            if not (offsets[token_start_index][0] <= start_char and offsets[token_end_index][1] >= end_char):
                tokenized_examples["start_positions"].append(cls_index)
                tokenized_examples["end_positions"].append(cls_index)
            else:
                # Otherwise move the token_start_index and token_end_index to the two ends of the answer.
                # Note: we could go after the last offset if the answer is the last word (edge case).
                while token_start_index < len(offsets) and offsets[token_start_index][0] <= start_char:
                    token_start_index += 1
                tokenized_examples["start_positions"].append(token_start_index - 1)
                while offsets[token_end_index][1] >= end_char:
                    token_end_index -= 1
                tokenized_examples["end_positions"].append(token_end_index + 1)

    return tokenized_examples

In [ ]:
tokenized_train = train_dataset.map(prepare_train_features, batched=True, remove_columns=train_dataset.column_names)
tokenized_valid = valid_dataset.map(prepare_train_features, batched=True, remove_columns=valid_dataset.column_names)

In [ ]:
from transformers import AutoModelForQuestionAnswering, TrainingArguments, Trainer

model = AutoModelForQuestionAnswering.from_pretrained(model_name)

In [ ]:
# DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
# print("[INFO] training using {}".format(torch.cuda.get_device_name(0)))

In [ ]:
gc.collect()

In [ ]:
epoch=2
batch_size = 8
warmup_steps = 500
weight_decay = 0.01

args = TrainingArguments(
    output_dir='./results',
    evaluation_strategy = "epoch",
    save_strategy = "epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=epoch,
    warmup_steps = warmup_steps,
    weight_decay = weight_decay,
    load_best_model_at_end=True
)

In [ ]:
from transformers import default_data_collator
data_collator = default_data_collator

In [ ]:
trainer = Trainer(
    model,
    args=args,
    train_dataset=tokenized_train,
    eval_dataset =tokenized_valid,
    data_collator=data_collator,
    tokenizer=tokenizer
)

In [ ]:
torch.cuda.empty_cache()

In [ ]:
# trainer.train()

In [ ]:
# trainer.save_model("chaii-xlm-roberta-larg-model")

In [ ]:
trainer.evaluate()

In [ ]:
def prepare_validation_features(examples):
    tokenized_examples = tokenizer(
        examples["question" if pad_on_right else "context"],
        examples["context" if pad_on_right else "question"],
        truncation="only_second" if pad_on_right else "only_first",
        max_length=max_length,
        stride=doc_stride,
        return_overflowing_tokens=True,
        return_offsets_mapping=True,
        padding="max_length",
    )

    sample_mapping = tokenized_examples.pop("overflow_to_sample_mapping")
    tokenized_examples["example_id"] = []

    for i in range(len(tokenized_examples["input_ids"])):
        # Grab the sequence corresponding to that example (to know what is the context and what is the question).
        sequence_ids = tokenized_examples.sequence_ids(i)
        context_index = 1 if pad_on_right else 0

        sample_index = sample_mapping[i]
        tokenized_examples["example_id"].append(examples["id"][sample_index])

        tokenized_examples["offset_mapping"][i] = [
            (o if sequence_ids[k] == context_index else None)
            for k, o in enumerate(tokenized_examples["offset_mapping"][i])
        ]
    return tokenized_examples

In [ ]:
valid_features = valid_dataset.map(prepare_validation_features,batched=True,remove_columns=valid_dataset.column_names)

In [ ]:
import collections

examples = valid_dataset
features = valid_features
example_id_to_index = {k: i for i, k in enumerate(examples["id"])}
features_per_example = collections.defaultdict(list)

for i, feature in enumerate(features):
    features_per_example[example_id_to_index[feature["example_id"]]].append(i)

In [ ]:
gc.collect()

In [ ]:
from tqdm.auto import tqdm
squad_v2 = False

def postprocess_qa_predictions(examples, features, raw_predictions, n_best_size = 20, max_answer_length = 30):
    all_start_logits, all_end_logits = raw_predictions
    # Build a map example to its corresponding features.
    example_id_to_index = {k: i for i, k in enumerate(examples["id"])}
    features_per_example = collections.defaultdict(list)
    for i, feature in enumerate(features):
        features_per_example[example_id_to_index[feature["example_id"]]].append(i)

    # The dictionaries we have to fill.
    predictions = collections.OrderedDict()

    # Logging.
    print(f"Post-processing {len(examples)} example predictions split into {len(features)} features.")

    # Let's loop over all the examples!
    for example_index, example in enumerate(tqdm(examples)):
        # Those are the indices of the features associated to the current example.
        feature_indices = features_per_example[example_index]

        min_null_score = None # Only used if squad_v2 is True.
        valid_answers = []
        
        context = example["context"]
        # Looping through all the features associated to the current example.
        for feature_index in feature_indices:
            # We grab the predictions of the model for this feature.
            start_logits = all_start_logits[feature_index]
            end_logits = all_end_logits[feature_index]
            # This is what will allow us to map some the positions in our logits to span of texts in the original
            # context.
            offset_mapping = features[feature_index]["offset_mapping"]

            # Update minimum null prediction.
            cls_index = features[feature_index]["input_ids"].index(tokenizer.cls_token_id)
            feature_null_score = start_logits[cls_index] + end_logits[cls_index]
            if min_null_score is None or min_null_score < feature_null_score:
                min_null_score = feature_null_score

            # Go through all possibilities for the `n_best_size` greater start and end logits.
            start_indexes = np.argsort(start_logits)[-1 : -n_best_size - 1 : -1].tolist()
            end_indexes = np.argsort(end_logits)[-1 : -n_best_size - 1 : -1].tolist()
            for start_index in start_indexes:
                for end_index in end_indexes:
                    # Don't consider out-of-scope answers, either because the indices are out of bounds or correspond
                    # to part of the input_ids that are not in the context.
                    if (
                        start_index >= len(offset_mapping)
                        or end_index >= len(offset_mapping)
                        or offset_mapping[start_index] is None
                        or offset_mapping[end_index] is None
                    ):
                        continue
                    # Don't consider answers with a length that is either < 0 or > max_answer_length.
                    if end_index < start_index or end_index - start_index + 1 > max_answer_length:
                        continue

                    start_char = offset_mapping[start_index][0]
                    end_char = offset_mapping[end_index][1]
                    valid_answers.append(
                        {
                            "score": start_logits[start_index] + end_logits[end_index],
                            "text": context[start_char: end_char]
                        }
                    )
        
        if len(valid_answers) > 0:
            best_answer = sorted(valid_answers, key=lambda x: x["score"], reverse=True)[0]
        else:
            # In the very rare edge case we have not a single non-null prediction, we create a fake prediction to avoid
            # failure.
            best_answer = {"text": "", "score": 0.0}
        # Let's pick our final answer: the best one or the null answer (only for squad_v2)
        if not squad_v2:
            predictions[example["id"]] = best_answer["text"]
        else:
            answer = best_answer["text"] if best_answer["score"] > min_null_score else ""
            predictions[example["id"]] = answer
    
    return predictions

In [ ]:
valid_feats = valid_features.map(lambda example: example, remove_columns=['example_id', 'offset_mapping'])
raw_predictions = trainer.predict(valid_feats)

In [ ]:
final_predictions = postprocess_qa_predictions(valid_dataset, valid_features, raw_predictions.predictions)

In [ ]:
valid_preds = [{"id": ex["id"], "answer": ex["answers"]['text'][0]} for ex in valid_dataset]
pred_valid_df= pd.DataFrame(valid_preds)
pred_valid_df.head(1)

In [ ]:
def jaccard(row): 
    str1 = row[0]
    str2 = row[1]
    a = set(str1.lower().split()) 
    b = set(str2.lower().split())
    c = a.intersection(b)
    return float(len(c)) / (len(a) + len(b) - len(c))

In [ ]:
pred_valid_df['prediction'] = pred_valid_df['id'].apply(lambda r: final_predictions[r])
pred_valid_df['jaccard'] = pred_valid_df[['answer', 'prediction']].apply(jaccard, axis=1)
pred_valid_df.head()

In [ ]:
round(pred_valid_df.jaccard.mean(),2)

In [ ]:
test_dataset = Dataset.from_pandas(test)
test_features = test_dataset.map(
    prepare_validation_features,
    batched=True,
    remove_columns=test_dataset.column_names
)

In [ ]:
test_feats = test_features.map(lambda example: example, remove_columns=['example_id', 'offset_mapping'])


In [ ]:
raw_test_predictions = trainer.predict(test_feats)

In [ ]:
test_features.set_format(type=test_features.format["type"], columns=list(test_features.features.keys()))
final_test_predictions = postprocess_qa_predictions(test_dataset, test_features, raw_test_predictions.predictions)

In [ ]:
sub=pd.read_csv("../input/chaii-hindi-and-tamil-question-answering/sample_submission.csv")

In [ ]:
sub['PredictionString'] = sub['id'].apply(lambda x: final_test_predictions[x])

In [ ]:
##sub['Correct Answers'] = ['येलन','20 अप्रैल 2010','१२ मार्च १८२४','13','சுவாமிநாதன் மற்றும் வர்கீஸ் குரியன்']# As I get it manually.
##sub['jaccard score']   = sub[['PredictionString', 'Correct Answers']].apply(jaccard, axis=1)

In [ ]:
##round(sub['jaccard score'].mean(),2)

In [ ]:
print(sub['PredictionString'].head())

In [ ]:
##print(sub['Correct Answers'].head())

In [ ]:
sub.to_csv('submission.csv',index=False)

In [ ]:
sub